In [2]:
import datetime
import unittest

import genetic

In this fitness function we will return a gap to not punish cases that will stall the algorithm if it doesn't fullfit the next best solution. So in this case we will give partial credit for an almost good solution.

In [3]:
def get_fitness(genes):
    fitness = 1
    gap = 0

    for i in range(1, len(genes)):
        if genes[i] > genes[i - 1]:
            fitness += 1
        else:
            gap += genes[i - 1] - genes[i]
    return Fitness(fitness, gap)

Display the canddate, the fitness and the time

In [4]:
def display(candidate, startTime):
    timeDiff = datetime.datetime.now() - startTime
    print("{}\t=> {}\t{}".format(
        ', '.join(map(str, candidate.Genes)),
        candidate.Fitness,
        timeDiff))

We need to somehow prefer sequences with small gaps between the runs to those with large gaps. There are a couple of ways to do that. One way that is often seen when the engine has constraints on the type of the fitness value is to scale up the fitness value, multiplying by a large value like 1000 for example, and then subtracting the amount of the gap. Another way would be to make the fitness score a floating point value with the gap value in the decimal portion and subtracted from 1. These would both work but we then have to decode them either mentally or actually if we want to break out each part of the fitness value for display.
Lucky for us our engine doesn’t have a type limitation so we’re going to keep the values separate but encapsulated in a problem-specific fitness object that can be compared to its own type. This allows us to determine the better of two gene sequences based on their fitness, while making the fitness object responsible for how it is displayed.

In [5]:
class Fitness:
    def __init__(self, numbersInSequenceCount, totalGap):
        self.NumbersInSequenceCount = numbersInSequenceCount
        self.TotalGap = totalGap
    #Next we need to be able to compare two fitness values.

    def __gt__(self, other):
        if self.NumbersInSequenceCount != other.NumbersInSequenceCount:
            return self.NumbersInSequenceCount > other.NumbersInSequenceCount
        return self.TotalGap < other.TotalGap
    
    #Lastly, the Fitness object needs to know how to convert itself to a string for display:
    def __str__(self):
        return "{} Sequential, {} Total Gap".format(
            self.NumbersInSequenceCount,
            self.TotalGap)

In [6]:
class SortedNumbersTests(unittest.TestCase):
    def test_sort_10_numbers(self):
        self.sort_numbers(10)

    def sort_numbers(self, totalNumbers):
        geneset = [i for i in range(100)]
        startTime = datetime.datetime.now()

        def fnDisplay(candidate):
            display(candidate, startTime)

        def fnGetFitness(genes):
            return get_fitness(genes)

        optimalFitness = Fitness(totalNumbers, 0)
        best = genetic.get_best(fnGetFitness, totalNumbers, optimalFitness,
                                geneset, fnDisplay)
        self.assertTrue(not optimalFitness > best.Fitness)

    def test_benchmark(self):
        genetic.Benchmark.run(lambda: self.sort_numbers(40))

In [7]:
SortedNumbersTests().test_sort_10_numbers()

40, 98, 69, 49, 72, 80, 95, 57, 81, 2	=> 6 Sequential, 166 Total Gap	0:00:00.001809
40, 98, 69, 49, 72, 80, 95, 60, 81, 2	=> 6 Sequential, 163 Total Gap	0:00:00.002125
40, 53, 69, 49, 72, 80, 95, 60, 81, 2	=> 7 Sequential, 134 Total Gap	0:00:00.002238
40, 53, 69, 49, 72, 80, 95, 60, 81, 10	=> 7 Sequential, 126 Total Gap	0:00:00.002432
40, 53, 69, 49, 57, 80, 95, 60, 76, 10	=> 7 Sequential, 121 Total Gap	0:00:00.002631
40, 57, 69, 49, 57, 67, 44, 60, 76, 10	=> 7 Sequential, 109 Total Gap	0:00:00.002861
14, 57, 69, 49, 57, 67, 44, 60, 76, 62	=> 7 Sequential, 57 Total Gap	0:00:00.003008
14, 57, 69, 49, 57, 67, 67, 70, 76, 62	=> 7 Sequential, 34 Total Gap	0:00:00.003462
41, 57, 69, 53, 57, 67, 67, 70, 76, 62	=> 7 Sequential, 30 Total Gap	0:00:00.003992
41, 57, 69, 53, 57, 67, 67, 70, 76, 66	=> 7 Sequential, 26 Total Gap	0:00:00.004589
37, 57, 69, 53, 57, 67, 67, 70, 76, 93	=> 8 Sequential, 16 Total Gap	0:00:00.005551
37, 57, 69, 53, 57, 66, 67, 70, 76, 99	=> 9 Sequential, 16 Total Gap	0:00